# 집계함수에 따른 파생변수 추가 생성 

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

%matplotlib inline
warnings.filterwarnings('ignore')

train = pd.read_csv('train_features.csv')
train_label = pd.read_csv('train_labels.csv')
test = pd.read_csv('test_features.csv')
submission = pd.read_csv('sample_submission.csv')

In [6]:
pd.set_option('display.max_columns', None)

In [7]:
def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_

In [12]:
tmp = train.drop('time', axis = 1, inplace = False)
X = tmp.groupby('id').agg([np.max, np.min, np.mean, 
                       np.var, np.std, np.median,
                      percentile(25), percentile(75)])

In [16]:
X

acc_x                                                    \
          amax      amin      mean       var       std    median   
id                                                                 
0     1.344268  0.591940  0.931329  0.036664  0.191479  0.956149   
1     1.234020 -2.156208 -0.766580  0.245548  0.495528 -0.805767   
2     1.219836 -1.142847  0.039836  0.506904  0.711972  0.140667   
3    -0.622250 -1.417751 -0.887702  0.017134  0.130899 -0.880343   
4     0.599720 -2.429109 -0.659018  0.245193  0.495170 -0.941146   
...        ...       ...       ...       ...       ...       ...   
3120  0.390798 -1.624711 -0.300454  0.162550  0.403175 -0.105704   
3121 -0.446650 -1.575455 -0.974298  0.028887  0.169963 -0.980053   
3122  0.744666 -2.578974 -1.114246  0.467567  0.683789 -1.057063   
3123  0.915846 -0.929133 -0.111333  0.187248  0.432722 -0.178023   
3124  0.538809 -1.013813 -0.434048  0.272596  0.522107 -0.613512   

                                     acc_y                                \
     percentile_25 percentile_75      amax      amin      mean       var   
id                                                                         
0         0.747302      1.080077  0.176871 -0.624113 -0.218471  0.031375   
1        -1.057583     -0.383457  0.700065 -1.295598 -0.317258  0.113175   
2        -0.688672      0.736468  0.650645 -0.690990 -0.082403  0.021646   
3        -0.982072     -0.784699  0.283721 -0.540827 -0.087668  0.037639   
4        -0.960309     -0.347234  1.724782 -2.055076 -0.337067  0.325247   
...            ...           ...       ...       ...       ...       ...   
3120     -0.588712     -0.086355  0.168070 -1.289257 -0.669209  0.035006   
3121     -1.021473     -0.909872  0.117965 -0.609743 -0.233373  0.013767   
3122     -1.697727     -0.569290  1.268138 -2.036646 -0.362196  0.112211   
3123     -0.458019      0.223488  1.473727  0.272406  0.880362  0.046994   
3124     -0.935463      0.089389 -0.193247 -0.952761 -0.623010  0.043506   

                                                         acc_z            \
           std    median percentile_25 percentile_75      amax      amin   
id                                                                         
0     0.177131 -0.240638     -0.358621     -0.067206 -0.054876 -0.786336   
1     0.336415 -0.228905     -0.607455     -0.065757  0.888661 -1.019531   
2     0.147127 -0.062598     -0.135405     -0.007626  1.332992  0.073846   
3     0.194008 -0.054577     -0.219341      0.068742  0.598814 -0.257124   
4     0.570305 -0.168467     -0.899829      0.090496  2.678034 -1.250483   
...        ...       ...           ...           ...       ...       ...   
3120  0.187098 -0.737803     -0.754197     -0.590281  0.849966 -0.551185   
3121  0.117334 -0.208131     -0.297183     -0.169748  0.218621 -0.403201   
3122  0.334978 -0.306436     -0.601282     -0.127102  0.803950 -0.459267   
3123  0.216780  0.902584      0.763079      1.010501  0.382916 -0.646863   
3124  0.208582 -0.677878     -0.815802     -0.418824  0.543977 -0.380657   

                                                                          \
          mean       var       std    median percentile_25 percentile_75   
id                                                                         
0    -0.370422  0.018260  0.135131 -0.346749     -0.450826     -0.272323   
1    -0.004223  0.249396  0.499395 -0.034583     -0.299597      0.436215   
2     0.626012  0.061905  0.248807  0.634781      0.399984      0.827337   
3     0.227357  0.042387  0.205882  0.231537      0.057929      0.449981   
4     0.202758  0.151824  0.389646  0.293556      0.138963      0.333143   
...        ...       ...       ...       ...           ...           ...   
3120  0.335934  0.117825  0.343256  0.530820      0.124946      0.548821   
3121 -0.073771  0.007417  0.086122 -0.084534     -0.117581     -0.022301   
3122  0.241518  0.044597  0.211179  0.242813      0.114727      0.378710   
3123 -0.122757

In [19]:
test_F = test.drop('time', axis = 1, inplace = False).groupby('id').agg([np.max, np.min, np.mean, 
                                                                         np.var, np.std, np.median,
                                                                        percentile(25), percentile(75)])

In [10]:
y = train_label['label']

In [17]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import time
import datetime

In [13]:
X_train, X_test , y_train, y_test = train_test_split(X, y, test_size = 0.2, 
                                                    random_state = 42)
print( y_train.nunique(), y_test.nunique())

61 61


In [14]:
evals = [(X_test, y_test)]

In [15]:
xgb = XGBClassifier(n_estimators = 400, learning_rate = 0.05,
                    max_depth = 8, objective = 'multi:softprob',random_state = 0)                    

In [18]:
start = time.time()
xgb.fit(X, y, eval_metric = 'mlogloss', eval_set = evals,
       early_stopping_rounds = 100)

sec = time.time() - start

times = str(datetime.timedelta(seconds = sec)).split('.')
print('소요시간 : ', times)

[0]	validation_0-mlogloss:3.34965
[1]	validation_0-mlogloss:3.02477
[2]	validation_0-mlogloss:2.77445
[3]	validation_0-mlogloss:2.57190
[4]	validation_0-mlogloss:2.39866
[5]	validation_0-mlogloss:2.24984
[6]	validation_0-mlogloss:2.12067
[7]	validation_0-mlogloss:2.00381
[8]	validation_0-mlogloss:1.89903
[9]	validation_0-mlogloss:1.80218
[10]	validation_0-mlogloss:1.71520
[11]	validation_0-mlogloss:1.63489
[12]	validation_0-mlogloss:1.55881
[13]	validation_0-mlogloss:1.48863
[14]	validation_0-mlogloss:1.42395
[15]	validation_0-mlogloss:1.36273
[16]	validation_0-mlogloss:1.30468
[17]	validation_0-mlogloss:1.24992
[18]	validation_0-mlogloss:1.19809
[19]	validation_0-mlogloss:1.14989
[20]	validation_0-mlogloss:1.10377
[21]	validation_0-mlogloss:1.06031
[22]	validation_0-mlogloss:1.01844
[23]	validation_0-mlogloss:0.97920
[24]	validation_0-mlogloss:0.94214
[25]	validation_0-mlogloss:0.90659
[26]	validation_0-mlogloss:0.87274
[27]	validation_0-mlogloss:0.84073
[28]	validation_0-mlogloss:0.8

In [20]:
y_pred = xgb.predict_proba(test_F)

In [22]:
y_pred.shape

(782, 61)

In [23]:
submission.iloc[:,1:]=y_pred
submission.to_csv('submission파일/xgb_add_agg.csv', index = False)

In [25]:
from sklearn.metrics import log_loss

In [26]:
log_loss(y_test, xgb.predict_proba(X_test))

0.013711367387115024